In [1]:
import warnings
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from torch.utils.data import DataLoader, TensorDataset
import torch
from tqdm import tqdm
import gc


# Suppressing unnecessary warnings
warnings.filterwarnings("ignore")

2023-11-13 16:08:30.428295: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-13 16:08:30.467504: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-13 16:08:30.467529: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-13 16:08:30.467553: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-13 16:08:30.476035: I tensorflow/core/platform/cpu_feature_g

In [ ]:

# Check if a GPU is available, and if so, use it
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#torch.cuda.empty_cache()
#gc.collect()    

#tokenizer = T5Tokenizer.from_pretrained("./T5Snapshot/tokenizer_")
#model = T5ForConditionalGeneration.from_pretrained("./T5Snapshot/model").to(device)

tokenizer = T5Tokenizer.from_pretrained("t5-small") # When training from scratch
model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)  # Move model to the GPU

# the following 2 hyperparameters are task-specific
max_source_length = 512  # Set to a value that covers most assembly code sequences
max_target_length = 512  # Set to a value that covers most C code sequences


#max_split_size_mb = 1024  # Adjust this value to a lower value if needed
#tokenizer.max_split_size_mb = max_split_size_mb

# Read input sequences from assembly.txt
with open("assembly.txt", "r", encoding="utf-8") as assembly_file:
    assembly_lines = assembly_file.readlines()

# Read output sequences from functions.txt
with open("function.txt", "r", encoding="utf-8") as functions_file:
    functions_lines = functions_file.readlines()

# Ensure that the number of lines in assembly.txt and functions.txt are the same
assert len(assembly_lines) == len(functions_lines), "Number of lines in assembly.txt and functions.txt must be the same."

# Encode inputs and targets
input_sequences = [assembly_line for assembly_line in assembly_lines]
output_sequences = [function_line for function_line in functions_lines]

encoding = tokenizer(
    input_sequences,
    padding="longest",
    max_length=max_source_length,
    truncation=True,
    return_tensors="pt",
)

input_ids, attention_mask = encoding.input_ids.to(device), encoding.attention_mask.to(device)

print(model.device)  # Print the device the model is on
print(input_ids.device)  # Print the device of input_ids

target_encoding = tokenizer(
    output_sequences,
    padding="longest",
    max_length=max_target_length,
    truncation=True,
    return_tensors="pt",
)
labels = target_encoding.input_ids.to(device)

# Replace padding token id's of the labels by -100 so it's ignored by the loss
labels[labels == tokenizer.pad_token_id] = -100

# Set hyperparameters
learning_rate = 5e-5
batch_size = 16 #thumb of rule
num_epochs = 20
num_warmup_steps = 0.1 * num_epochs * len(input_sequences) / batch_size

# Define optimizer with GPU device
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Create DataLoader for training data
train_dataset = TensorDataset(input_ids, attention_mask, labels)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Training loop on the GPU
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}"):
        input_ids_batch, attention_mask_batch, labels_batch = batch

        # Forward pass
        loss = model(
            input_ids=input_ids_batch,
            attention_mask=attention_mask_batch,
            labels=labels_batch
        ).loss

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}, Average Loss: {avg_loss}")
    model.save_pretrained(f"./T5Snapshot/model_epoch{epoch + 1}_loss{avg_loss:.2f}")
    tokenizer.save_pretrained(f"./T5Snapshot/tokenizer_epoch{epoch + 1}_loss{avg_loss:.2f}")

# Save the trained model
model.save_pretrained("./T5Snapshot/model")
tokenizer.save_pretrained("./T5Snapshot/tokenizer")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


cuda:0
cuda:0


Epoch 1: 100%|██████████| 2258/2258 [14:54<00:00,  2.53it/s]


Epoch 1, Average Loss: 2.6865308898755993


Epoch 2: 100%|██████████| 2258/2258 [14:55<00:00,  2.52it/s]


Epoch 2, Average Loss: 2.0189411313371175


Epoch 3:   7%|▋         | 169/2258 [01:06<13:46,  2.53it/s]

**Inference**

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = T5Tokenizer.from_pretrained("./T5Snapshot/tokenizer_epoch1_loss1.51",local_files_only=True)
model = T5ForConditionalGeneration.from_pretrained("./T5Snapshot/model_epoch1_loss1.51",local_files_only=True).to(device)

input_sequence = 'endbr64 ;push   rbp;mov    rbp,rsp;mov    DWORD PTR [rbp-(1)],edi;mov    DWORD PTR [rbp-(0)],esi;mov    edx,DWORD PTR [rbp-(1)];mov    eax,DWORD PTR [rbp-(0)];add    eax,edx;pop    rbp;ret'
input_ids = tokenizer(input_sequence, return_tensors="pt").input_ids.to(device)
outputs = model.generate(input_ids,max_length=300, num_return_sequences=1)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))